<a href="https://colab.research.google.com/github/914book/FinalProjectGroupD/blob/Circle_DB/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from sqlalchemy import create_engine
from getpass import getpass
secret = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [2]:
pip install psycopg2

In [16]:
import psycopg2
connection = psycopg2.connect(
    host = 'housingdatabase.chkn2wwlsbys.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = secret,
    database='NYChouses'
    )
cursor=connection.cursor()


In [17]:
import pandas as pd

sql = """
SELECT *
FROM fulloutpuhousing
"""
fulloutpuhousing = pd.read_sql(sql, con=connection)

In [18]:
pd.set_option('display.max_colwidth',1000)

In [19]:
#table for EDA
fulloutpuhousing.head()


,address,zipcode,date,beds,bath,sq_ft,house_price,city,county,county_type,boro,income_over_100k,income_under_100k,employed,unemployed
0,"4205 48th Ave APT 3G, Flushing, NY 11377",11377,2020-10-16,Studio,1.0,53400,244000,Woodside,Woodside,Queens,Queens County,166650,1015040,1099996,39401
1,"14404 76th Rd, Flushing, NY 11367",11367,2020-10-16,3,3.0,1980,680000,Flushing,Flushing,Queens,Queens County,166650,1015040,1099996,39401
2,"35 88th St, Brooklyn, NY 11209",11209,2020-10-16,3,1.0,1520,900000,Brooklyn,Brooklyn,Kings,Kings County (Brooklyn),188640,1067440,1149461,48699
3,"11128 130th St, Jamaica, NY 11420",11420,2020-10-16,3,2.0,1440,625000,South Ozone Park,South Ozone Park,Queens,Queens County,166650,1015040,1099996,39401
4,"20102 32nd Ave, Flushing, NY 11361",11361,2020-10-16,2,1.0,1406,1720000,Bayside,Bayside,Queens,Queens County,166650,1015040,1099996,39401


In [20]:
#table for M/L model based of test data share with Toral
house2data = fulloutpuhousing[['house_price', 'address', 'zipcode', 'date', 'beds', 'bath','sq_ft', 'boro', 'employed', 'unemployed', 'income_over_100k', 'income_under_100k']]
house2data.columns = ['price','address','zipcode','Date','beds','bath','sq_ft','boro','Employed','Unemployed','income above 100k','income below 100k']
house2data


,price,address,zipcode,Date,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k
0,244000,"4205 48th Ave APT 3G, Flushing, NY 11377",11377,2020-10-16,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040
1,680000,"14404 76th Rd, Flushing, NY 11367",11367,2020-10-16,3,3.0,1980,Queens County,1099996,39401,166650,1015040
2,900000,"35 88th St, Brooklyn, NY 11209",11209,2020-10-16,3,1.0,1520,Kings County (Brooklyn),1149461,48699,188640,1067440
3,625000,"11128 130th St, Jamaica, NY 11420",11420,2020-10-16,3,2.0,1440,Queens County,1099996,39401,166650,1015040
4,1720000,"20102 32nd Ave, Flushing, NY 11361",11361,2020-10-16,2,1.0,1406,Queens County,1099996,39401,166650,1015040
...,...,...,...,...,...,...,...,...,...,...,...,...
24209,167310,"84-70 129th St, Kew Gardens, NY 11415",11415,2017-10-19,1,1.0,800,Queens County,1099996,39401,166650,1015040
24210,545000,"9721 129th St, Jamaica, NY 11419",11419,2017-10-19,1,1.0,750,Queens County,1099996,39401,166650,1015040
24211,620000,"8731 116th St, Jamaica, NY 11418",11418,2017-10-18,5,0.0,2025,Queens County,1099996,39401,166650,1015040
24212,790000,"4030 247th St, Flushing, NY 11363",11363,2017-10-18,3,1.0,1738,Queens County,1099996,39401,166650,1015040
